In [17]:
import os
import numpy as np
import talib as ta
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [44]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"

In [1]:
filelist = os.listdir(path)
filelist.sort()
df = pd.DataFrame()
for i in tqdm(filelist[-300:]):
    df0 = pd.read_csv(os.path.join(path, i))
    df = pd.concat([df, df0])
df

NameError: name 'os' is not defined

In [46]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()
df1

,t,o,h,l,c,v
0,1670976000000,17777.1,17833.9,17749.5,17817.8,10803.908
1,1670979600000,17817.8,17844.9,17772.3,17781.4,6999.399
2,1670983200000,17781.4,17815.3,17753.1,17795.1,5767.537
3,1670986800000,17795.1,17839.6,17782.4,17782.4,7150.765
4,1670990400000,17782.4,17790.5,17752.1,17772.1,5255.493
...,...,...,...,...,...,...
19,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886
20,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344
21,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588
22,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446


In [47]:
# 计算ALMA
def calculate_alma(df, data='c', length=9, offset=0.85, sigma=6):
    """

    :param df: 要处理的数据
    :param c: 要处理的列名
    :param length: 滑动窗口的长度（周期）
    :param offset: 偏移量一般取0.85
    :param sigma: 一般取6
    :return: df
    """
    # 计算中间变量
    m = (length - 1) * offset  # 6.8
    s = length / sigma  #2.25
    wi = np.exp(-(np.arange(length) - m) ** 2 / (2 * s ** 2))
    df[f'alma_{length}'] = df[data].rolling(length).apply(lambda window: np.dot(window.astype(np.float64), wi.T), raw=True) / np.sum(wi,dtype=np.float64)
    return df


df1 = calculate_alma(df1, data='c', length=9, offset=0.85, sigma=6)
df1 = calculate_alma(df1, data='c', length=16, offset=0.85, sigma=6)
df1 = df1[16:]
df1

,t,o,h,l,c,v,alma_9,alma_16
16,1671033600000,18054.8,18149.6,18053.1,18096.9,37273.480,18028.985222,17957.867743
17,1671037200000,18096.9,18146.6,18069.0,18089.3,16822.793,18064.218083,17997.582908
18,1671040800000,18089.4,18400.0,17805.0,18215.0,101592.610,18109.315389,18046.194577
19,1671044400000,18215.3,18400.0,17704.3,17789.5,145336.062,18059.511713,18038.147489
20,1671048000000,17789.5,17944.4,17659.4,17762.1,63340.245,17959.930710,18005.909727
...,...,...,...,...,...,...,...,...
19,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,27542.174388,27504.524567
20,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,27564.800682,27522.984405
21,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,27579.975763,27545.316360
22,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,27583.480620,27560.710464


In [48]:
df1.to_csv('./ALMAH.csv', index=False)

,t,o,h,l,c,v
0,1.648685e+12,47053.1,47250.0,46950.0,47139.2,5257.439
1,1.648688e+12,47139.1,47408.7,47125.0,47380.1,5953.960
2,1.648692e+12,47380.1,47398.2,47161.2,47206.1,4451.067
3,1.648696e+12,47206.2,47235.5,47056.0,47088.4,4150.407
4,1.648699e+12,47088.3,47143.3,46980.0,47011.8,5371.315
...,...,...,...,...,...,...
19,1.672168e+12,16630.9,16681.3,16580.0,16681.3,16782.160
20,1.672171e+12,16681.3,16689.9,16651.0,16658.3,9923.424
21,1.672175e+12,16658.4,16714.1,16646.3,16696.3,7955.232
22,1.672178e+12,16696.4,16702.0,16678.5,16682.0,5519.179
